# Training the model on the full data set
to make every wine accessible, we train the model again on the full dataset

In [97]:
from IPython.display import display, clear_output
import os
import seaborn as sns
import numpy as np
from importlib import reload
# import random
import pickle
# import time
# import matplotlib.pyplot as plt
# import itertools
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
# from sklearn.metrics import confusion_matrix

# from scipy.spatial.distance import cosine
# from nltk.corpus import words
# import gensim

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from nltk.tokenize import word_tokenize

# from collections import Counter
%matplotlib inline
from wine_and_cheese_utils import WineList, df_columns, regexp_tokenizer

import warnings
    
warnings.filterwarnings("ignore")

In [2]:
wl = WineList(file='cleaned')


In [99]:
# collect the tagged document object and save a dict of tags/set
tagged_data = [TaggedDocument(words=wl.tokenize(row.description), tags=[row.Index]) for row in wl.df.itertuples()]
file_name = "tagged_data_set.pkl"

if os.path.exists(file_name):
    with open(file_name, 'rb') as f:
        tagged_data_set = pickle.load(f)
else:
    tagged_data_set = dict(zip([x.tags[0] for x in tagged_data], [set(x.words) for x in tagged_data]))
    with open(file_name, 'wb') as f:
            pickle.dump(tagged_data_set, f, pickle.HIGHEST_PROTOCOL)

In [4]:
retrain = False

column_name = 'region_1'
# load or train the model
model_file_name = 'doc2vec_on_{}_no_region_variety_full_dataset.model'.format(column_name)
if not os.path.exists(model_file_name) or retrain:
    max_epochs = 20
    vec_size = 40
    alpha = 0.025

    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha, 
    #                 min_alpha=0.00025,
                    min_count=10,
                    dm =0)

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        clear_output(wait=True)
        display('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save(model_file_name)
else:
    model = Doc2Vec.load(model_file_name)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [93]:
# all(elem in list1  for elem in list2)
# %timeit ok = [all(elem in ["lime","white","peach"] for elem in x.words) for x in tagged_data]
# test = wl.df.description.str.contains("black-currant")
# test2=pd.Series(index=[x.tags[0] for x in tagged_data])
desc = ["lime"] 
def get_description_match_series(desc):
    indexes = index=[x.tags[0] for x in tagged_data]
    data=[all(elem in desc for elem in x.words) for x in tagged_data]
    return data
#     return pd.Series(data=[all(elem in desc for elem in x.words) for x in tagged_data],index=indexes)
%timeit get_description_match_series(desc)

def get_description_match_series_from_sets(desc,tagged_data_set):
    set_desc = set(desc)
    indexes = index=[x.tags[0] for x in tagged_data]
    data=[set_desc.issubset(this_set) for this_set in tagged_data_set.values()]
    return data
#     return pd.Series(data=[all(elem in desc for elem in x.words) for x in tagged_data],index=indexes)
%timeit get_description_match_series_from_sets(desc,tagged_data_set)

103 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
44.7 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


['tart',
 'snappy',
 'flavors',
 'lime',
 'flesh',
 'rind',
 'dominate',
 'green',
 'pineapple',
 'pokes',
 'crisp',
 'acidity',
 'underscoring',
 'flavors',
 'wine',
 'stainless',
 'steel',
 'fermented']

TypeError: accuracy() missing 1 required positional argument: 'questions'

In [32]:
a = set([1,3,4,43])
a.add(6)

In [33]:
a

{1, 3, 4, 6, 43}

In [35]:
%timeit 'chill' in model.wv.vocab

115 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
